# Recurrent Neural Networks

- 給定一個語句，判斷他有沒有惡意（負面標 1，正面標 0）
- 參考李弘毅老師ML課程[HW4](https://colab.research.google.com/drive/16d1Xox0OW-VNuxDn1pvy2UXFIPfieCb9#scrollTo=ICDIhhgCY2-M)


### Change Working Dir & Some Initial Configure

In [49]:
import os
# from google.colab import drive
# drive.mount('/content/drive')
# path_prefix = '/content/drive/My Drive/Colab Notebooks/hw4 - Recurrent Neural Network'
# path_prefix = './'
# %cd '/content/drive/My Drive/Colab Notebooks/hw4 - Recurrent Neural Network/'
# %cd C:\Users\a5465\Desktop\side-project\ChatBot
os.getcwd()


'C:\\Users\\a5465\\desktop\\side-project\\ChatBot'

In [50]:
# this is for filtering the warnings
import warnings
import os
warnings.filterwarnings('ignore')

# path prefix setting
model_prefix = './model'
if not os.path.exists('./model'):
  os.mkdir('./model')
path_prefix = './'
data_prefix = './data'

embd_dim = 250

### Utils

In [23]:
# # utils.py
# # 這個 block 用來先定義一些等等常用到的函式
# import torch
# import numpy as np
# import pandas as pd
# import torch.optim as optim
# import torch.nn.functional as F

# def load_training_data(path=train_x_path):
#     # 把 training 時需要的 data 讀進來
#     # 如果是 'training_label.txt'，需要讀取 label，如果是 'training_nolabel.txt'，不需要讀取 label
#     if 'KaoKao' in path:
#         with open(path, 'r', encoding='utf8') as f:
#           lines = f.readlines()
#           lines = [line.strip('\n').split(' ') for line in lines]
#         x = [line[2:] for line in lines]
#         for i in range(len(x)):
#           while True:
#             try:
#               x[i].remove('')
#             except:
#               break

#         y = [line[0] for line in lines]
#         return x, y
#     # else:
#     #     with open(path, 'r') as f:
#     #         lines = f.readlines()
#     #         x = [line.strip('\n').split(' ') for line in lines]
#     #     return x

# def load_testing_data(path='testing_data'):
#     # 把 testing 時需要的 data 讀進來
#     with open(path, 'r') as f:
#         lines = f.readlines()
#         X = ["".join(line.strip('\n').split(",")[1:]).strip() for line in lines[1:]]
#         X = [sen.split(' ') for sen in X]
#     return X

# def evaluation(outputs, labels):
#     # outputs => probability (float)
#     # labels => labels
#     # print('outputs.shape: {}'.format(outputs.shape))
#     # print('labels.shape: {}'.format(labels.shape))
#     # outputs[outputs>=0.5] = 1 # 大於等於 0.5 為正面
#     # outputs[outputs<0.5] = 0 # 小於 0.5 為負面
#     outputs_class = torch.zeros(labels.shape[0]).to(device)
#     for i in range(0, labels.shape[0]):
#       max_idx = 0
#       for j in range(1, outputs[i].shape[0]):
#         if outputs[i,j] > outputs[i,max_idx]:
#           max_idx = j
#       outputs_class[i] = max_idx
#     correct = torch.sum(torch.eq(outputs_class, labels)).item()
#     return correct

In [51]:
# utils.py
# 這個 block 用來先定義一些等等常用到的函式
import torch
import numpy as np
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
import jieba

def load_training_data():
    # load positive sentences
    lines_cut = []
    labels = []
    with open(os.path.join(data_prefix, 'positive.txt').replace('\\','/'), 'r', encoding='utf8') as f:
      lines = f.readlines()
      lines = [line.strip('\n') for line in lines]
      for line in lines:
        jieba_cut = jieba.cut(line)
        line_cut = []
        for word in jieba_cut:
          line_cut.append(word)
        lines_cut.append(line_cut)
        labels.append(1)

    # load negative sentences
    with open(os.path.join(data_prefix, 'negative.txt').replace('\\','/'), 'r', encoding='utf8') as f:
      lines = f.readlines()
      lines = [line.strip('\n') for line in lines]
      for line in lines:
        jieba_cut = jieba.cut(line)
        line_cut = []
        for word in jieba_cut:
          line_cut.append(word)
        lines_cut.append(line_cut)
        labels.append(0)

    with open(os.path.join(data_prefix, 'ntusd-negative.txt').replace('\\','/'), 'r', encoding='utf8') as f:
      lines = f.readlines()
      lines = [line.strip('\n') for line in lines]
      for line in lines:
        jieba_cut = jieba.cut(line)
        line_cut = []
        for word in jieba_cut:
          line_cut.append(word)
        lines_cut.append(line_cut)
        labels.append(2)


    return lines_cut, labels

def loading_corpus:
# def load_testing_data(path='testing_data'):
#     # 把 testing 時需要的 data 讀進來
#     with open(path, 'r') as f:
#         lines = f.readlines()
#         X = ["".join(line.strip('\n').split(",")[1:]).strip() for line in lines[1:]]
#         X = [sen.split(' ') for sen in X]
#     return X

def evaluation(outputs, labels):
    # outputs => probability (float)
    # labels => labels
    # print('outputs.shape: {}'.format(outputs.shape))
    # print('labels.shape: {}'.format(labels.shape))
    # outputs[outputs>=0.5] = 1 # 大於等於 0.5 為正面
    # outputs[outputs<0.5] = 0 # 小於 0.5 為負面
    outputs_class = torch.zeros(labels.shape[0]).to(device)
    for i in range(0, labels.shape[0]):
      max_idx = 0
      for j in range(1, outputs[i].shape[0]):
        if outputs[i,j] > outputs[i,max_idx]:
          max_idx = j
      outputs_class[i] = max_idx
    correct = torch.sum(torch.eq(outputs_class, labels)).item()
    return correct

### Train Word to Vector

In [76]:
# w2v.py
# 這個 block 是用來訓練 word to vector 的 word embedding
# 注意！這個 block 在訓練 word to vector 時是用 cpu，可能要花到 10 分鐘以上
import os
import numpy as np
import pandas as pd
import argparse
from gensim.models import word2vec

def train_word2vec(x):
    # 訓練 word to vector 的 word embedding
    model = word2vec.Word2Vec(x, corpus_file=x, size=embd_dim, window=5, min_count=5, workers=12, iter =10, sg=1)
    return model

if __name__ == "__main__":
    print("loading training data ...")
    train_x, y = load_training_data()
    # train_x_no_label = load_training_data('training_nolabel.txt')

    # print("loading testing data ...")
    # test_x = load_testing_data('testing_data.txt')

    #model = train_word2vec(train_x + train_x_no_label + test_x)
    model = train_word2vec(train_x)
    
    print("saving model ...")
    # model.save(os.path.join(path_prefix, 'model/w2v_all.model'))
    model.save(os.path.join(model_prefix, 'w2v_all-NTCH.model').replace('\\','/'))

loading training data ...


RuntimeError: ignored

In [53]:
print(train_x[-5:])
print(y[-5:])

[['噁', '心'], ['噁', '心鬼'], ['憋死'], ['憋', '著'], ['蹩腳']]
[2, 2, 2, 2, 2]


In [54]:
sen_max = 0
sum = 0
for sen in train_x:
  sum += len(sen)
  # print(sen)
  if len(sen) > sen_max:
    sen_max = len(sen)
sen_avg = sum/len(train_x)
print('senLen_max: {}'.format(sen_max))
print('senLen_avg: {}'.format(sen_avg))

senLen_max: 8710
senLen_avg: 32.7176577443838


### Data Preprocess

In [55]:
# preprocess.py
# 這個 block 用來做 data 的預處理
from torch import nn
from gensim.models import Word2Vec

class Preprocess():
    def __init__(self, sentences, sen_len, w2v_path):
        self.w2v_path = w2v_path
        self.sentences = sentences
        self.sen_len = sen_len
        self.idx2word = []
        self.word2idx = {}
        self.embedding_matrix = []
    def get_w2v_model(self):
        # 把之前訓練好的 word to vec 模型讀進來
        self.embedding = Word2Vec.load(self.w2v_path)
        self.embedding_dim = self.embedding.vector_size
    def add_embedding(self, word):
        # 把 word 加進 embedding，並賦予他一個隨機生成的 representation vector
        # word 只會是 "<PAD>" 或 "<UNK>"
        vector = torch.empty(1, self.embedding_dim)
        torch.nn.init.uniform_(vector)
        self.word2idx[word] = len(self.word2idx)
        self.idx2word.append(word)
        self.embedding_matrix = torch.cat([self.embedding_matrix, vector], 0)
    def make_embedding(self, load=True):
        print("Get embedding ...")
        # 取得訓練好的 Word2vec word embedding
        if load:
            print("loading word to vec model ...")
            self.get_w2v_model()
        else:
            raise NotImplementedError
        # 製作一個 word2idx 的 dictionary
        # 製作一個 idx2word 的 list
        # 製作一個 word2vector 的 list
        for i, word in enumerate(self.embedding.wv.vocab):
            print('get words #{}'.format(i+1), end='\r')
            #e.g. self.word2index['he'] = 1 
            #e.g. self.index2word[1] = 'he'
            #e.g. self.vectors[1] = 'he' vector
            self.word2idx[word] = len(self.word2idx)
            self.idx2word.append(word)
            self.embedding_matrix.append(self.embedding[word])
        print('')
        self.embedding_matrix = torch.tensor(self.embedding_matrix)
        # 將 "<PAD>" 跟 "<UNK>" 加進 embedding 裡面
        self.add_embedding("<PAD>")
        self.add_embedding("<UNK>")
        print("total words: {}".format(len(self.embedding_matrix)))
        return self.embedding_matrix
    def pad_sequence(self, sentence):
        # 將每個句子變成一樣的長度
        if len(sentence) > self.sen_len:
            sentence = sentence[:self.sen_len]
        else:
            pad_len = self.sen_len - len(sentence)
            for _ in range(pad_len):
                sentence.append(self.word2idx["<PAD>"])
        assert len(sentence) == self.sen_len
        return sentence
    def sentence_word2idx(self):
        # 把句子裡面的字轉成相對應的 index
        sentence_list = []
        for i, sen in enumerate(self.sentences):
            print('sentence count #{}'.format(i+1), end='\r')
            sentence_idx = []
            for word in sen:
                if (word in self.word2idx.keys()):
                    sentence_idx.append(self.word2idx[word])
                else:
                    sentence_idx.append(self.word2idx["<UNK>"])
            # 將每個句子變成一樣的長度
            sentence_idx = self.pad_sequence(sentence_idx)
            sentence_list.append(sentence_idx)
        return torch.LongTensor(sentence_list)
    def labels_to_tensor(self, y):
        # 把 labels 轉成 tensor
        y = [int(label) for label in y]
        return torch.LongTensor(y)


### Custom Dataset Implementation


In [56]:
# data.py
# 實作了 dataset 所需要的 '__init__', '__getitem__', '__len__'
# 好讓 dataloader 能使用
import torch
from torch.utils import data

class CustomDataset(data.Dataset):
    """
    Expected data shape like:(data_num, data_len)
    Data can be a list of numpy array or a list of lists
    input data shape : (data_num, seq_len, feature_dim)
    
    __len__ will return the number of data
    """
    def __init__(self, X, y):
        self.data = X
        self.label = y
    def __getitem__(self, idx):
        if self.label is None: return self.data[idx]
        return self.data[idx], self.label[idx]
    def __len__(self):
        return len(self.data)

### Model

In [57]:
# model.py
# 這個 block 是要拿來訓練的模型
import torch
from torch import nn
class LSTM_Net(nn.Module):
    def __init__(self, embedding, embedding_dim, hidden_dim, num_layers, dropout=0.5, fix_embedding=True):
        super(LSTM_Net, self).__init__()
        # 製作 embedding layer
        self.embedding = torch.nn.Embedding(embedding.size(0),embedding.size(1))
        self.embedding.weight = torch.nn.Parameter(embedding)
        # 是否將 embedding fix 住，如果 fix_embedding 為 False，在訓練過程中，embedding 也會跟著被訓練
        self.embedding.weight.requires_grad = False if fix_embedding else True
        self.embedding_dim = embedding.size(1)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.classifier = nn.Sequential( nn.Dropout(dropout),
                            nn.Linear(hidden_dim, 2),
                            nn.Sigmoid() )
    def forward(self, inputs):
        inputs = self.embedding(inputs)
        x, _ = self.lstm(inputs, None)
        # x 的 dimension (batch, seq_len, hidden_size)
        # 取用 LSTM 最後一層的 hidden state
        x = x[:, -1, :] 
        x = self.classifier(x)
        return x

### Train

In [58]:
# train.py
# 這個 block 是用來訓練模型的
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

def training(batch_size, n_epoch, lr, model_dir, train, valid, model, device):
    # total = sum(p.numel() for p in model.parameters())
    # trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    # print('\nstart training, parameter total:{}, trainable:{}\n'.format(total, trainable))
    model.train() # 將 model 的模式設為 train，這樣 optimizer 就可以更新 model 的參數
    criterion = nn.CrossEntropyLoss().to(device) # 定義損失函數，這裡我們使用 binary cross entropy loss
    t_batch = len(train) 
    v_batch = len(valid) 
    optimizer = optim.Adam(model.parameters(), lr=lr) # 將模型的參數給 optimizer，並給予適當的 learning rate
    total_loss, total_acc, best_acc = 0, 0, 0
    for epoch in range(n_epoch):
        print('\nepoch: {}'.format(epoch+1))
        total_loss, total_acc = 0, 0
        # 這段做 training
        for i, (inputs, labels) in enumerate(train):
            inputs = inputs.to(device, dtype=torch.long) # device 為 "cuda"，將 inputs 轉成 torch.cuda.LongTensor
            labels = labels.to(device, dtype=torch.float) # device為 "cuda"，將 labels 轉成 torch.cuda.FloatTensor，因為等等要餵進 criterion，所以型態要是 float
            optimizer.zero_grad() # 由於 loss.backward() 的 gradient 會累加，所以每次餵完一個 batch 後需要歸零
            outputs = model(inputs) # 將 input 餵給模型
            # outputs = outputs.squeeze() # 去掉最外面的 dimension，好讓 outputs 可以餵進 criterion()
            # print('outputs.shape: {}'.format(outputs.shape))
            loss = criterion(outputs.double(), labels.long()).to(device) # 計算此時模型的 training loss
            # print('outputs: {}'.format(outputs))
            # print('labels: {}'.format(labels))
            # print('loss: {}'.format(loss.item()))
            # input()
            loss.backward() # 算 loss 的 gradient
            optimizer.step() # 更新訓練模型的參數
            correct = evaluation(outputs, labels) # 計算此時模型的 training accuracy
            total_acc += (correct / batch_size)
            total_loss += loss.item()
            # print('[ Epoch{}: {}/{} ] loss:{:.3f} acc:{:.3f} '.format(epoch+1, i+1, t_batch, loss.item(), correct*100/batch_size))
        print('Train | Loss:{:.5f} Acc: {:.3f}'.format(total_loss/t_batch, total_acc/t_batch*100))

        # 這段做 validation
        model.eval() # 將 model 的模式設為 eval，這樣 model 的參數就會固定住
        with torch.no_grad():
            total_loss, total_acc = 0, 0
            for i, (inputs, labels) in enumerate(valid):
                inputs = inputs.to(device, dtype=torch.long) # device 為 "cuda"，將 inputs 轉成 torch.cuda.LongTensor
                labels = labels.to(device, dtype=torch.float) # device 為 "cuda"，將 labels 轉成 torch.cuda.FloatTensor，因為等等要餵進 criterion，所以型態要是 float
                outputs = model(inputs) # 將 input 餵給模型
                # outputs = outputs.squeeze() # 去掉最外面的 dimension，好讓 outputs 可以餵進 criterion()
                # print('inputs.shape: {}'.format(inputs.shape))
                # print('outputs.shape: {}'.format(outputs.shape))
                loss = criterion(outputs.double(), labels.long()).to(device) # 計算此時模型的 validation loss
                correct = evaluation(outputs, labels) # 計算此時模型的 validation accuracy
                total_acc += (correct / batch_size)
                total_loss += loss.item()

            print("Valid | Loss:{:.5f} Acc: {:.3f} ".format(total_loss/v_batch, total_acc/v_batch*100))
            if total_acc > best_acc:
                # 如果 validation 的結果優於之前所有的結果，就把當下的模型存下來以備之後做預測時使用
                best_acc = total_acc
                #torch.save(model, "{}/val_acc_{:.3f}.model".format(model_dir,total_acc/v_batch*100))
                torch.save(model, "{}/ckpt-NTCH.model".format(model_dir))
                print('saving model with acc {:.3f}'.format(total_acc/v_batch*100))
        print('-----------------------------------------------')
        model.train() # 將 model 的模式設為 train，這樣 optimizer 就可以更新 model 的參數（因為剛剛轉成 eval 模式）

    # outputs = model(train.dataset.data.to(device))
    # print(outputs)
    # return outputs

### Test

In [59]:
# test.py
# 這個 block 用來對 testing_data.txt 做預測
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

def testing(batch_size, test_loader, model, device):
    model.eval()
    ret_output = []
    with torch.no_grad():
        for i, inputs in enumerate(test_loader):
            inputs = inputs.to(device, dtype=torch.long)
            outputs = model(inputs)
            outputs = outputs.squeeze()
            ret_output.append(outputs)

    
    return ret_output

### Main

In [75]:
# main.py
import os
import torch
import argparse
import numpy as np
from torch import nn
from gensim.models import word2vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# 通過 torch.cuda.is_available() 的回傳值進行判斷是否有使用 GPU 的環境，如果有的話 device 就設為 "cuda"，沒有的話就設為 "cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 處理好各個 data 的路徑
# train_with_label = os.path.join(path_prefix, train_data_path)
# train_no_label = os.path.join(path_prefix, 'training_nolabel.txt')
# testing_data = os.path.join(path_prefix, 'testing_data.txt')

w2v_path = os.path.join(model_prefix, 'w2v_all-NTCH.model').replace('\\','/') # 處理 word to vec model 的路徑

# 定義句子長度、要不要固定 embedding、batch 大小、要訓練幾個 epoch、learning rate 的值、model 的資料夾路徑
sen_len = 47
fix_embedding = True # fix embedding during training
batch_size = 256
epoch = 10
lr = 0.01
# model_dir = os.path.join(path_prefix, 'model/') # model directory for checkpoint model
model_dir = model_prefix # model directory for checkpoint model

print("loading data ...") # 把 'training_label.txt' 跟 'training_nolabel.txt' 讀進來
train_x, y = load_training_data()
# train_x_no_label = load_training_data()

# 對 input 跟 labels 做預處理
preprocess = Preprocess(train_x, sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
train_x = preprocess.sentence_word2idx()
y = preprocess.labels_to_tensor(y)

# 製作一個 model 的對象
model = LSTM_Net(embedding, embedding_dim=embd_dim, hidden_dim=150, num_layers=1, dropout=0.5, fix_embedding=fix_embedding)
model = model.to(device) # device為 "cuda"，model 使用 GPU 來訓練（餵進去的 inputs 也需要是 cuda tensor）

# 把 data 分為 training data 跟 validation data（將一部份 training data 拿去當作 validation data）

df_orgin = pd.DataFrame(np.concatenate((train_x.numpy(), y.numpy().reshape(y.numpy().shape[0],1)), axis=1))
df = df_orgin[df_orgin[47] == 0].sample(n=df_orgin[df_orgin[47] == 1].shape[0])
df = pd.concat([df, df_orgin[df_orgin[47] == 1]])
df = pd.concat([df, df_orgin[df_orgin[47] == 2]])
df[47] = df[47].replace(2,0)
train, val = train_test_split(df, test_size=0.2, shuffle=True)
y_train = train.iloc[:,-1].values
y_val = val.iloc[:,-1].values
X_train = train.iloc[:,:-1].values
X_val = val.iloc[:,:-1].values

X_train = torch.from_numpy(X_train).to(device)
y_train = torch.from_numpy(y_train).to(device)
X_val = torch.from_numpy(X_val).to(device)
y_val = torch.from_numpy(y_val).to(device)

# train_num = 30000
# X_train, X_val, y_train, y_val = train_x[:train_num], train_x[train_num:], y[:train_num], y[train_num:]

# 把 data 做成 dataset 供 dataloader 取用
train_dataset = CustomDataset(X=X_train, y=y_train)
val_dataset = CustomDataset(X=X_val, y=y_val)

# 把 data 轉成 batch of tensors
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
                        # num_workers = 2)

val_loader = torch.utils.data.DataLoader(dataset = val_dataset, batch_size = batch_size, shuffle = False)
                        # num_workers = 8)

# 開始訓練
training(batch_size, epoch, lr, model_dir, train_loader, val_loader, model, device)



loading data ...
Get embedding ...
loading word to vec model ...
get words #52287
total words: 52289

epoch: 1
Train | Loss:0.50410 Acc: 80.171
Valid | Loss:0.43273 Acc: 87.580 
saving model with acc 87.580
-----------------------------------------------

epoch: 2
Train | Loss:0.43509 Acc: 87.494
Valid | Loss:0.41940 Acc: 89.045 
saving model with acc 89.045
-----------------------------------------------

epoch: 3
Train | Loss:0.41973 Acc: 89.043
Valid | Loss:0.42128 Acc: 88.749 
-----------------------------------------------

epoch: 4
Train | Loss:0.41589 Acc: 89.423
Valid | Loss:0.41669 Acc: 89.300 
saving model with acc 89.300
-----------------------------------------------

epoch: 5
Train | Loss:0.41552 Acc: 89.506
Valid | Loss:0.41499 Acc: 89.471 
saving model with acc 89.471
-----------------------------------------------

epoch: 6
Train | Loss:0.41318 Acc: 89.725
Valid | Loss:0.41406 Acc: 89.491 
saving model with acc 89.491
-----------------------------------------------

epo

In [61]:
# fod idx in 
df[47] = df[47].replace(0,2)

In [62]:
df

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
289320,71119,151133,71120,14476,9147,267911,267911,267911,267911,267911,...,267911,267911,267911,267911,267911,267911,267911,267911,267911,2
81076,8352,14,205,561,641,38,267911,267911,267911,267911,...,267911,267911,267911,267911,267911,267911,267911,267911,267911,2
232075,150,1941,374,267911,267911,267911,267911,267911,267911,267911,...,267911,267911,267911,267911,267911,267911,267911,267911,267911,2
116559,14,1305,83753,956,38,267911,267911,267911,267911,267911,...,267911,267911,267911,267911,267911,267911,267911,267911,267911,2
179568,94043,127877,267911,267911,267911,267911,267911,267911,267911,267911,...,267911,267911,267911,267911,267911,267911,267911,267911,267911,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367726,17078,1169,267911,267911,267911,267911,267911,267911,267911,267911,...,267911,267911,267911,267911,267911,267911,267911,267911,267911,2
367727,17078,56132,267911,267911,267911,267911,267911,267911,267911,267911,...,267911,267911,267911,267911,267911,267911,267911,267911,267911,2
367728,242056,267911,267911,267911,267911,267911,267911,267911,267911,267911,...,267911,267911,267911,267911,267911,267911,267911,267911,267911,2
367729,2237,1038,267911,267911,267911,267911,267911,267911,267911,267911,...,267911,267911,267911,267911,267911,267911,267911,267911,267911,2


In [63]:
outputs_class = []
for i in range(outputs.shape[0]):
  if outputs[i,0] > outputs[i,1]:
    outputs_class.append(0)
  else:
    outputs_class.append(1)

AttributeError: ignored

In [ ]:
cm = confusion_matrix(outputs_class, train_loader.dataset.label.cpu())
print(cm)
print(accuracy_score(outputs_class, train_loader.dataset.label.cpu()))

In [ ]:
sum = 0
for y in y_train:
  if y == 1:
    sum += 1
print(sum)

In [73]:
# Find the closer word
preprocess.embedding.wv.closer_than('厲害','好棒')

['也',
 '不',
 '真的',
 '讓',
 '想',
 '什麼',
 '去',
 '做',
 '多',
 '話',
 '喜歡',
 '事情',
 '事',
 '難過',
 '一樣',
 '一次',
 '像',
 '很多',
 '一點',
 '哭',
 '開心',
 '幫',
 '大',
 '需要',
 '.....',
 '問題',
 '懂',
 '最近',
 '錯',
 '痛',
 '打',
 '感謝',
 '不好',
 '長',
 '起來',
 '痛苦',
 '電話',
 '恨',
 '這次',
 '認識',
 '明明',
 '累',
 'my',
 '重要',
 '英文',
 '唉',
 '難',
 '不用',
 '祝福',
 '2',
 '身體',
 '討厭',
 '女生',
 '付出',
 '擔心',
 '第一次',
 '捨',
 '玩',
 '真正',
 '了解',
 '清楚',
 '站',
 '出去',
 '騙',
 '健康',
 '不夠',
 '這裡',
 '壓力',
 '存在',
 '身上',
 '男人',
 '忙',
 '真心',
 '堅強',
 '聊天',
 '辛苦',
 '除了',
 '住',
 '容易',
 '方式',
 '值得',
 '在乎',
 '簡單',
 '自私',
 '見面',
 '回去',
 '聯絡',
 '男生',
 '天氣',
 '幫忙',
 '女朋友',
 '遠',
 '啥',
 '一開始',
 '受傷',
 '平安',
 '傻',
 '吵架',
 '字',
 '當下',
 '輕',
 'wretch',
 '面前',
 'sorry',
 '待',
 '懂得',
 '在意',
 '文章',
 '差',
 '靠',
 '因此',
 '兒子',
 '可惜',
 '成熟',
 '自以',
 '還好',
 '個性',
 '不行',
 '比較',
 '害',
 '沒事',
 '穿',
 '過的',
 '浪費',
 '打給',
 'XD',
 '高興',
 '在家',
 '沒用',
 '祝',
 '自',
 '失敗',
 '失戀',
 '高',
 '弄',
 '徹底',
 '冷',
 '講話',
 '令人',
 '那邊',
 'imgur',
 '單純',
 '喝',
 '幾次',
 '不錯',
 '舒服',
 '笨',
 

In [ ]:
train_loader.dataset.label[0:100]

### Predict and Write to csv file

In [ ]:
# # 開始測試模型並做預測
# print("loading testing data ...")
# test_x_word = load_testing_data(testing_data)
# preprocess = Preprocess(test_x_word, sen_len, w2v_path=w2v_path)
# embedding = preprocess.make_embedding(load=True)
# test_x = preprocess.sentence_word2idx()
# test_dataset = CustomDataset(X=test_x, y=None)
# test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
#                                             batch_size = batch_size,
#                                             shuffle = False,
#                                             num_workers = 8)
# print('\nload model ...')
# model = torch.load(os.path.join(model_dir, 'ckpt-Kao.model'))
# outputs = testing(batch_size, test_loader, model, device)

# # 寫到 csv 檔案供上傳 Kaggle
# tmp = pd.DataFrame({"id":[str(i) for i in range(len(test_x))],"label":outputs})
# print("save csv ...")
# tmp.to_csv(os.path.join(path_prefix, 'predict.csv'), index=False)
# print("Finish Predicting")

# # 以下是使用 command line 上傳到 Kaggle 的方式
# # 需要先 pip install kaggle、Create API Token，詳細請看 https://github.com/Kaggle/kaggle-api 以及 https://www.kaggle.com/code1110/how-to-submit-from-google-colab
# # kaggle competitions submit [competition-name] -f [csv file path]] -m [message]
# # e.g., kaggle competitions submit ml-2020spring-hw4 -f output/predict.csv -m "......"

In [66]:
preprocess = Preprocess([['輝煌']], sen_len, w2v_path=w2v_path)
embedding = preprocess.make_embedding(load=True)
test_x = preprocess.sentence_word2idx()
test_dataset = CustomDataset(X=test_x, y=None)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                        batch_size = batch_size,
                        shuffle = False)
                        # num_workers = 8)
print('\nload model ...')
model = torch.load(os.path.join(model_dir, 'ckpt-NTCH.model'))
outputs = testing(batch_size, test_loader, model, device)
print(outputs)

Get embedding ...
loading word to vec model ...
get words #267911
total words: 267913
sentence count #1
load model ...
[tensor([9.9980e-01, 1.7410e-06], device='cuda:0')]


#### Download the files to your computer

In [ ]:
# from google.colab import files
# files.download('predict.csv')